<div align="right"><i>Peter Norvig<br>2012; updated August 2019</i></div>

# Weighing Twelve Balls on a Balance Scale 

> *You are given twelve identical-looking balls and a two-sided scale. One of the balls is of a different weight, although you don't know whether it's lighter or heavier. How can you use just three weighings of the scale to determine not only what the different ball is, but also whether it's lighter or heavier?*

This is a traditional brain-teaser puzzle, meant to be solved with paper and pencil. 
But I want to solve not just this specific puzzle, but related puzzles where you can vary (a) the number of balls, (b) the number of weighings allowed, and (c) whether the odd ball might be heavier, lighter, or either, or neither. For that I'll need a program. (I originally wrote this program in 2012, but am republishing it here in revised form because the problem was mentioned in the [538 Riddler](https://fivethirtyeight.com/features/which-billiard-ball-is-rigged/) for 16 August 2019. It also appeared in a [Numberplay](https://wordplay.blogs.nytimes.com/2014/07/21/12coin/) column in 2014 (with coins instead of balls) among other venues.)

<p><center>
    🎱🎱🎱🎱⚖🎱🎱🎱🎱
</center>

# Design

Here are the concepts I'm dealing with:

- **balls**: In the general case I have N balls. I'll represent them with a list like `[1, 2, 3]` for N = 3.
- **oddballs**: Exactly one of the balls is **odd** in its weight. 
I'll represent the situation where ball N is heavier as +N, and where ball N is lighter as -N. (I'll represent the situation where no ball is odd with `0`; that's not needed for the puzzle stated above, but is a nice extension.) With N = 3, the set of all possible oddballs is `{+1, -1, +2, -2, +3, -3}`. 
- **puzzle**: The puzzle stated above can be expressed with `Puzzle(N=12, weighings=3, oddities={-1, +1})`. The third argument can be any subset of `{-1, 0, +1}`, where `-1` means "any one ball might be lighter," `+1` means "any one ball might be heavier," and `0` means "allow the possibility that no ball is odd; they all weigh the same." 
- **weighing**: I can weigh a collection of balls on the left versus a collection on the right, and the result will be that the left side is greater than, equal to, or less than the right in weight. I'll denote that with the call `weigh(L, R, oddball)`, which returns a string, `'gt'`, `'eq'`, or `'lt'`.
- **weight**: I'll arbitrarily say that a normal ball weighs 100, a lighter ball 99, and a heavier ball 101.
- **solution**: Given a puzzle, a solution is a **strategy tree** that can correctly discover the odd ball, whatever it is, in the allowable number of weighings.
- **strategy tree**: A tree with oddballs (integers) as leaf nodes, and interior nodes with 5 components:  `Tree(L, R, gt, eq, lt)` is a node where `L` and `R` are the collections of balls to be weighed, and `gt`, `eq`, and `lt` are subtrees for the three possible results of the weighing.
- **following a path in a tree**: I'll use `follow(tree, oddball)` to say *follow the path through the tree, doing each weighing under the assumption of the given oddball, and return the leaf node reached by the path&mdash;the oddball that the tree predicts.* Note that the function `follow` gets to see what the oddball is, but the `tree` never gets direct access to that; the tree has to figure it out by doing weighings.
- **valid tree**: a tree is a valid solution to a puzzle if no branch uses more than the allowable number of weighings, and if, for every possible oddball in the puzzle, following the path through the tree correctly returns that oddball as the answer. I'll use `valid(tree, puzzle)` for this.


# Implementation

Let's start implementing the concepts:

In [1]:
from collections import namedtuple
import random

#### Types

Ball = Oddball = int

class Puzzle:
    "Represent a specific ball-weighing puzzle."
    def __init__(self, N=12, weighings=3, oddities={-1, +1}):
        self.weighings = weighings
        self.balls     = list(range(1, N + 1))
        self.oddballs  = {b * o for b in self.balls for o in oddities}
        
Tree = namedtuple('Tree', 'L, R, gt, eq, lt')

#### Functions
    
def weigh(L, R, oddball) -> str:
    "Weigh balls L against balls R, given the oddball; return 'gt', 'eq', or 'lt'."
    diff = sum(weight(b, oddball) for b in L) - sum(weight(b, oddball) for b in R)
    return ('gt' if diff > 0 else
            'lt' if diff < 0 else
            'eq')

def weight(ball, oddball) -> int: 
    return 101 if +ball == oddball else 99 if -ball == oddball else 100
    
def solve(puzzle) -> Tree or None:
    "Return a valid tree; one that solves the puzzle, or None."
    tree = find_tree(puzzle, puzzle.oddballs, puzzle.weighings)
    return tree if valid(tree, puzzle) else None
    
def follow(tree, oddball) -> Oddball:
    "Follow a path through the tree and return the oddball that the tree leads us to."
    if isinstance(tree, Oddball):
        return tree
    else:
        result = weigh(tree.L, tree.R, oddball)
        return follow(getattr(tree, result), oddball)
    
def valid(tree, puzzle) -> bool:
    "Does the strategy tree solve the puzzle correctly for all possible oddballs?"
    return (tree is not None and
            depth(tree) <= puzzle.weighings and 
            all(follow(tree, oddball) == oddball 
                for oddball in puzzle.oddballs))

def depth(tree) -> int:
    "Maximum depth of a strategy tree."
    return (0 if isinstance(tree, Oddball) else 
            1 + max(depth(tree.gt), depth(tree.eq), depth(tree.lt)))

Let's try out some of these functions:

In [2]:
p8 = Puzzle(8) 

p8.weighings, p8.balls, p8.oddballs

(3,
 [1, 2, 3, 4, 5, 6, 7, 8],
 {-8, -7, -6, -5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7, 8})

In [3]:
# If we weigh balls 1, 2 against 3, 4, and the oddball is that 5 is lighter, the result should be 'eq'
weigh([1, 2], [3, 4], -5)

'eq'

In [4]:
weight(1, oddball=-5)

100

In [5]:
weight(5, oddball=-5)

99

# Strategy for Finding a Valid Tree

Now for the tricky part. We want to find a valid tree to solve a puzzle. The key idea is that a **weighing** gives us information by making a **partition** of the possible **oddballs** into entries for eaach of the three possible weighing results: `gt`, `eq`, or `lt`. Subsequent subtrees can handle each of the partitions.


In [6]:
def partition(L, R, oddballs) -> dict:
    "Build a dict of the possible outcomes (oddballs) from weighing L versus R."
    part = dict(gt=set(), eq=set(), lt=set())
    for odd in oddballs:
        part[weigh(L, R, odd)].add(odd)
    return part

For example, with 12 balls, if we weigh balls 1 and 2 on the left versus 11 and 12 on the right, then there are 4 ways  the left side can be greater than the right: either 1 or 2 is heavier or 11 or 12 is lighter. Similarly there are 4 ways of getting a `lt` weighing result. The remaining 16 possible oddballs&mdash;balls 3 through 10 being either heavier or lighter&mdash;show up in the `eq` entry of the partition:

In [7]:
p12 = Puzzle(12, 3)

partition([1, 2], [11, 12], p12.oddballs)

{'gt': {-12, -11, 1, 2},
 'eq': {-10, -9, -8, -7, -6, -5, -4, -3, 3, 4, 5, 6, 7, 8, 9, 10},
 'lt': {-2, -1, 11, 12}}

If this was the first weighing in our strategy tree, could we go on to solve the puzzle in 3 weighings? **No!** Any one weighing can at best partition the remaining possibilities into 3 equal entries. To solve the puzzle, we need every path in the tree to end up with only one possibility. So any two weighings can handle at most  3 &times; 3 = 9 possibilities; here we have 16, which is too many. We call this a **bad partition**.

The following is a **good partition** because each of the entries has 8 possibilities, and 8 is less than 9. (Note: being a good partition does not guarantee that the problem is solvable from there; but being a bad partition guarantees that it is not solvable.)

In [8]:
partition([1, 2, 3, 4], [9, 10, 11, 12], p12.oddballs)

{'gt': {-12, -11, -10, -9, 1, 2, 3, 4},
 'eq': {-8, -7, -6, -5, 5, 6, 7, 8},
 'lt': {-4, -3, -2, -1, 9, 10, 11, 12}}

So now we have a viable approach to implementing `find_tree`:

   - We call `find_tree(puzzle, oddballs, weighings)`. At the top level, the oddballs and number of weighings come from the puzzle. At recursive levels, we will reduce the number of oddball possibilities according to the partition, and the number of remaining weighings by 1 each time.
   - At each step we will randomly select two groups of balls, `L` and `R`, to be weighed.
   - We will then see what partition `L` and `R` gives us, and whether the partition is good or bad.
   - If the partition is bad, try another random selection of `L` and `R`.
   - If the partition is good, recursively try to find solution trees for each entry in the partition. If we can do that, we're done!
   - If we can't find a valid tree return `None`.

In [9]:
def find_tree(puzzle, oddballs, weighings) -> Tree or Oddball or None:
    "Find a strategy tree that covers all the oddballs in the given number of weighings."
    if len(oddballs) == 1:
        return oddballs.pop() # One oddball possibility left; we're done: leaf node
    elif len(oddballs) == 0:
        return 0              # No oddball
    elif weighings == 0:
        return None           # Can't find a solution in the allowable weighings
    else:
        for L, R, part in good_partitions(puzzle, oddballs, weighings - 1):
            subtrees = {r: find_tree(puzzle, part[r], weighings - 1) for r in part}
            if None not in subtrees.values(): 
                return Tree(L, R, **subtrees)
    
def good_partitions(puzzle, oddballs, weighings):
    "Yield (L, R, partition) such that no partition entry is longer than 3**weighings."
    for _ in range(1000): 
        L, R = random_LR(puzzle, oddballs)
        part = partition(L, R, oddballs)
        if max(map(len, part.values())) <= 3 ** weighings:
            yield L, R, part

def random_LR(puzzle, oddballs) -> (list, list):
    "Random choice of balls for L and R side."
    # Pick a random number of balls, B, then randomly pick B balls for each side.
    B = random.choice(range(1, (len(puzzle.balls) - 1) // 3 + 2))
    random.shuffle(puzzle.balls) 
    return puzzle.balls[:B], puzzle.balls[-B:]

Here we see that the subfunction `good_partitions` does its job:

In [10]:
next(good_partitions(p12, p12.oddballs, 2))

([5, 6, 9, 2],
 [10, 7, 4, 1],
 {'gt': {-10, -7, -4, -1, 2, 5, 6, 9},
  'eq': {-12, -11, -8, -3, 3, 8, 11, 12},
  'lt': {-9, -6, -5, -2, 1, 4, 7, 10}})

It uses `random`, so it won't get the same result every time:

In [11]:
next(good_partitions(p12, p12.oddballs, 2))

([10, 6, 12, 7],
 [1, 5, 9, 11],
 {'gt': {-11, -9, -5, -1, 6, 7, 10, 12},
  'eq': {-8, -4, -3, -2, 2, 3, 4, 8},
  'lt': {-12, -10, -7, -6, 1, 5, 9, 11}})

Of course, it would be great if `good_partitions` yielded *all* the good partitions, and even better if it didn't bother with redundant partitions. But that would be complicated, so instead I just have it choose random sets of balls for the left and right side, do that 1,000 times, and if I don't find a solution by then, it probably means there is none. That haphazard approach has worked so far, but I would certainly want to revisit `good_partitions` if I wanted to handle puzzles with hundreds or thousands of balls.

I'll introduce some useful functions: `random_LR` chooses a random selection of balls for the left and right side. It makes sure there are the same number on each side, and that that number, `B`, is between 1 and 1/3 of the total number of balls (rounded up).


# Solving Some Puzzles

Now we're ready to solve puzzles!

In [12]:
solve(p12)

Tree(L=[3, 2, 12, 7], R=[1, 10, 8, 9], gt=Tree(L=[10, 3, 5, 4], R=[2, 8, 9, 7], gt=Tree(L=[8, 5], R=[9, 4], gt=-9, eq=3, lt=-8), eq=Tree(L=[9, 3, 6], R=[12, 8, 1], gt=-1, eq=0, lt=12), lt=Tree(L=[12, 7, 10, 6], R=[4, 11, 9, 8], gt=7, eq=2, lt=-10)), eq=Tree(L=[10, 11, 4], R=[2, 12, 5], gt=Tree(L=[5, 1, 4], R=[8, 6, 12], gt=4, eq=11, lt=-5), eq=Tree(L=[6, 2, 1], R=[7, 5, 3], gt=6, eq=0, lt=-6), lt=Tree(L=[10, 11], R=[1, 4], gt=-4, eq=5, lt=-11)), lt=Tree(L=[10, 6, 3, 8], R=[11, 5, 1, 12], gt=Tree(L=[3, 2, 9, 4], R=[10, 11, 7, 12], gt=-12, eq=8, lt=10), eq=Tree(L=[5, 10], R=[9, 2], gt=-2, eq=-7, lt=9), lt=Tree(L=[11, 1], R=[10, 5], gt=1, eq=-3, lt=0)))

OK, that's hard to read&mdash;my bad. Let's look at a much easier puzzle: 3 balls, 1 weighing allowed, and the odd ball can only be lighter:

In [13]:
tree = solve(Puzzle(3, 1, {-1}))
tree

Tree(L=[3], R=[2], gt=-2, eq=-1, lt=-3)

This tree says you weigh one ball against another (leaving the third unweighed), and the three possible weighing results tell you which of the three balls  is lighter. 



# Prettier Output

Let's make the output easier to read. I'll use `[1·2·3·4 ⚖ 9·10·11·12] ➔` to mean *"Weigh balls 1,2,3,4 versus 9,10,11,12 to get a result..."*. I'll indent each interior node in the tree, and I'll use `>:` to mean *the result when the left hand side is greater than the right in weight is...*  

Also, note that at the top node of a tree, there's no sense randomly shuffling the balls&mdash;the only choice that matters is how many balls, `B`, to put on each side. Putting `1·2·3·4` on the left is no different than `3·7·9·12`, because each ball is undifferentiated at the start. I'll alter `random_LR` for this.

In [14]:
def do(puzzle):
    "Print the solution to the puzzle as indented text."
    print(indented(solve(puzzle)))
    
def indented(tree, i=0, prefix='') -> str:
    "Pretty, indented string representing a strategy tree."
    if isinstance(tree, Tree):
        subtrees = (indented(tree.gt, i+1, '>:'), indented(tree.eq, i+1, '=:'), indented(tree.lt, i+1, '<:'))
        indent   = ('' if i == 0 else ('\n' + ' ' * 5 * i))
        return f'{indent}{prefix}[{items(tree.L)} ⚖ {items(tree.R)}] ➔ {" ".join(subtrees)})'
    elif tree == 0 or tree == None:
        return f'{prefix}{tree}'
    else:
        return f'{prefix}{tree:+d}'
    
def items(collection) -> str: return '·'.join(map(str, sorted(collection)))

def random_LR(puzzle, oddballs) -> (list, list):
    "Random choice of balls for L and R side."
    # Pick a random number of balls, B, then pick B balls for each side.
    B = random.choice(range(1, (len(puzzle.balls) + 2) // 3 + 1))
    if oddballs == puzzle.oddballs:
        puzzle.balls.sort()
    else:
        random.shuffle(puzzle.balls) 
    return puzzle.balls[:B], puzzle.balls[-B:]

In [15]:
# 3 balls, 1 weighing, only lighter balls possible
do(Puzzle(3, 1, {-1}))

[1 ⚖ 3] ➔ >:-3 =:-2 <:-1)


In [16]:
# 3 balls, 2 weighings, lighter or heavier balls possible
do(Puzzle(3, 2))

[1 ⚖ 3] ➔ 
     >:[1 ⚖ 2] ➔ >:+1 =:-3 <:0) 
     =:[3 ⚖ 2] ➔ >:-2 =:0 <:+2) 
     <:[1 ⚖ 2] ➔ >:0 =:+3 <:-1))


In [17]:
# The original puzzle with 12 balls
do(p12)

[1·2·3·4 ⚖ 9·10·11·12] ➔ 
     >:[3·5·8·12 ⚖ 1·6·9·10] ➔ 
          >:[3·10 ⚖ 2·4] ➔ >:+3 =:-9 <:-10) 
          =:[2·3·6·10 ⚖ 4·8·9·12] ➔ >:+2 =:-11 <:+4) 
          <:[7·10·12 ⚖ 2·4·5] ➔ >:0 =:+1 <:-12)) 
     =:[2·3·6·11 ⚖ 1·7·8·12] ➔ 
          >:[1·4·11·12 ⚖ 3·6·7·9] ➔ >:-7 =:-8 <:+6) 
          =:[3·7·10·12 ⚖ 5·8·9·11] ➔ >:-5 =:0 <:+5) 
          <:[1·2·6·8 ⚖ 3·4·9·11] ➔ >:+8 =:+7 <:-6)) 
     <:[2·5·10·12 ⚖ 1·7·8·11] ➔ 
          >:[1·2·8·12 ⚖ 3·5·7·9] ➔ >:+12 =:+10 <:-1) 
          =:[1·7·11·12 ⚖ 4·5·8·9] ➔ >:-4 =:-3 <:+9) 
          <:[1·3·4·9 ⚖ 2·6·8·10] ➔ >:-2 =:+11 <:0)))


In [18]:
# A different solution to the same problem
do(p12)

[1·2·3·4 ⚖ 9·10·11·12] ➔ 
     >:[1·4·10 ⚖ 2·3·12] ➔ 
          >:[1·5·9 ⚖ 4·7·11] ➔ >:+1 =:-12 <:+4) 
          =:[3·10·11 ⚖ 1·8·9] ➔ >:-9 =:0 <:-11) 
          <:[3·9·11·12 ⚖ 1·2·7·8] ➔ >:+3 =:-10 <:+2)) 
     =:[2·5 ⚖ 6·7] ➔ 
          >:[4·5·7·10 ⚖ 1·8·9·12] ➔ >:+5 =:-6 <:-7) 
          =:[2·4·7 ⚖ 8·10·11] ➔ >:-8 =:0 <:+8) 
          <:[1·4·6·11 ⚖ 3·7·8·12] ➔ >:+6 =:-5 <:+7)) 
     <:[4·9·11 ⚖ 1·10·12] ➔ 
          >:[1·4·9 ⚖ 6·10·12] ➔ >:+9 =:+11 <:-1) 
          =:[1·9·10 ⚖ 2·7·12] ➔ >:-2 =:-3 <:0) 
          <:[5·6·8 ⚖ 2·4·10] ➔ >:-4 =:+12 <:+10)))


# Other Puzzles



We can do **12 balls in 3 weighings** even when we add in the possibility that all the balls weigh the same (we use `0` to denote this situation):

In [19]:
do(Puzzle(12, 3, {-1, 0, +1}))

[1·2·3·4 ⚖ 9·10·11·12] ➔ 
     >:[1·8·9·11 ⚖ 4·5·10·12] ➔ 
          >:[1·9·10·11 ⚖ 3·4·6·8] ➔ >:+1 =:-12 <:-10) 
          =:[1·2·7·12 ⚖ 4·5·6·10] ➔ >:+2 =:+3 <:0) 
          <:[4·9·12 ⚖ 3·6·7] ➔ >:+4 =:-11 <:-9)) 
     =:[1·5·6·12 ⚖ 4·8·9·11] ➔ 
          >:[2·4·6 ⚖ 1·5·7] ➔ >:+6 =:-8 <:+5) 
          =:[7·10 ⚖ 1·5] ➔ >:+7 =:0 <:-7) 
          <:[5·7·8·12 ⚖ 3·4·9·11] ➔ >:+8 =:-6 <:-5)) 
     <:[3·9·12 ⚖ 5·10·11] ➔ 
          >:[9 ⚖ 6] ➔ >:+9 =:+12 <:0) 
          =:[4·8·9 ⚖ 2·5·10] ➔ >:-2 =:-1 <:-4) 
          <:[2·3·11·12 ⚖ 4·5·8·9] ➔ >:+11 =:+10 <:-3)))


Can we solve the **13-balls in 3 weighings** problem, which has 26 possibilities? 

In [20]:
do(Puzzle(13))

None


**No**, and the reason is that there is no first weighing that partitions the 26 possibilities into 9/9/8; the best we can do is partition into 8/10/8 or 10/6/10, and 10 possibilities can't be distinguished in the remaining two weighings:

In [21]:
partition([1, 2, 3, 4], [10, 11, 12, 13], Puzzle(13).oddballs)

{'gt': {-13, -12, -11, -10, 1, 2, 3, 4},
 'eq': {-9, -8, -7, -6, -5, 5, 6, 7, 8, 9},
 'lt': {-4, -3, -2, -1, 10, 11, 12, 13}}

In [22]:
partition([1, 2, 3, 4, 5], [9, 10, 11, 12, 13], Puzzle(13).oddballs)

{'gt': {-13, -12, -11, -10, -9, 1, 2, 3, 4, 5},
 'eq': {-8, -7, -6, 6, 7, 8},
 'lt': {-5, -4, -3, -2, -1, 9, 10, 11, 12, 13}}

We can do **27 balls in 3 weighings** if we know that the odd ball can only be lighter, not heavier. And we can do **26 balls** under the condition that either one ball is lighter or all the balls weigh the same. In both cases there are 27 possibilities, the maximum number we can handle in 3 weighings.

In [23]:
do(Puzzle(27, 3, {-1}))

[1·2·3·4·5·6·7·8·9 ⚖ 19·20·21·22·23·24·25·26·27] ➔ 
     >:[1·5·7·10·11·13·19·21·23 ⚖ 2·9·14·15·17·18·20·24·27] ➔ 
          >:[1·6·14·16·19·21·22·24 ⚖ 2·9·11·12·18·20·25·26] ➔ >:-20 =:-27 <:-24) 
          =:[2·5·7·8·13·14·15·22 ⚖ 1·10·11·17·19·20·23·25] ➔ >:-25 =:-26 <:-22) 
          <:[6·8·9·13·17·18·19·22 ⚖ 4·7·10·11·20·21·24·27] ➔ >:-21 =:-23 <:-19)) 
     =:[2·6·7·8·10·12·13·22 ⚖ 3·9·11·14·17·19·25·27] ➔ 
          >:[5·11·13·16·23·25·27 ⚖ 2·4·6·9·17·20·21] ➔ >:-17 =:-14 <:-11) 
          =:[1·12·16·21·24 ⚖ 2·7·11·18·22] ➔ >:-18 =:-15 <:-16) 
          <:[2·4·10·15·17·19·20·23·27 ⚖ 3·5·7·8·12·16·18·21·24] ➔ >:-12 =:-13 <:-10)) 
     <:[1·3·4·10·12·15·18·25·27 ⚖ 6·8·9·14·19·20·22·23·26] ➔ 
          >:[3·6·14·15·17·26·27 ⚖ 2·8·11·16·19·21·25] ➔ >:-8 =:-9 <:-6) 
          =:[5·11·15·23 ⚖ 7·16·20·22] ➔ >:-7 =:-2 <:-5) 
          <:[1·8·11·12·20·25·26 ⚖ 4·6·7·10·18·22·23] ➔ >:-4 =:-3 <:-1)))


In [24]:
do(Puzzle(26, 3, {-1, 0}))

[1·2·3·4·5·6·7·8·9 ⚖ 18·19·20·21·22·23·24·25·26] ➔ 
     >:[3·6·8·11·15·21·23·24 ⚖ 1·2·5·13·14·19·20·22] ➔ 
          >:[3·4·6·10·11·20·24·25 ⚖ 5·7·8·9·13·14·21·22] ➔ >:-22 =:-19 <:-20) 
          =:[5·7·12·13·17·20·21·23·26 ⚖ 2·3·4·8·9·10·19·24·25] ➔ >:-25 =:-18 <:-26) 
          <:[3·6·13·21·22 ⚖ 8·10·18·19·24] ➔ >:-24 =:-23 <:-21)) 
     =:[6·7·9·12·13·15·18·24·25 ⚖ 2·4·8·10·14·17·20·22·26] ➔ 
          >:[8·17·20·26 ⚖ 14·16·21·24] ➔ >:-14 =:-10 <:-17) 
          =:[4·9·10·11·19 ⚖ 3·8·15·16·26] ➔ >:-16 =:0 <:-11) 
          <:[2·11·12·14·19·22·24·26 ⚖ 4·8·13·16·18·21·23·25] ➔ >:-13 =:-15 <:-12)) 
     <:[2·3·6·10·14·18·19·22·23 ⚖ 1·5·7·11·13·15·20·24·26] ➔ 
          >:[1·2·9·11·18·19 ⚖ 3·7·8·17·22·26] ➔ >:-7 =:-5 <:-1) 
          =:[7·8·10·11·16·18·19·21·22 ⚖ 1·2·3·9·14·15·17·23·24] ➔ >:-9 =:-4 <:-8) 
          <:[1·3·8·14·18·20·22·23·24 ⚖ 2·5·7·10·12·13·16·17·25] ➔ >:-2 =:-6 <:-3)))


Here's a puzzle with **25 balls** with an unusual set of possibilities: either one of the odd-numbered balls is heavier, or one of the even-numbered balls is lighter, or all the balls weigh the same. I can't describe that  with one call to `Puzzle`; I'll have to construct the set of oddballs separately:

In [25]:
p25 = Puzzle(25, 3)
p25.oddballs = {(+b if b % 2 else -b) for b in p25.balls} | {0}
print(p25.oddballs)

{0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, -24, -22, -20, -18, -16, -14, -12, -10, -8, -6, -4, -2}


In [26]:
do(p25)

[1·2·3·4·5·6·7·8·9 ⚖ 17·18·19·20·21·22·23·24·25] ➔ 
     >:[3·6·11·14·15·16·21·23 ⚖ 1·5·7·12·13·18·19·22] ➔ 
          >:[8·15·16·20·22·24·25 ⚖ 6·9·10·17·18·19·23] ➔ >:-18 =:+3 <:-22) 
          =:[4·5·13·14·17·23 ⚖ 7·9·11·16·20·22] ➔ >:-20 =:-24 <:+9) 
          <:[3·5·8·20·21·25 ⚖ 1·6·9·12·15·24] ➔ >:+5 =:+7 <:+1)) 
     =:[8·9·11·14·16·19·20·21·23 ⚖ 1·2·5·7·12·13·17·18·22] ➔ 
          >:[3·8·11·12·16·17·19 ⚖ 2·4·5·13·20·23·24] ➔ >:+11 =:0 <:-12) 
          =:[2·10·15·18·22 ⚖ 8·12·19·21·24] ➔ >:+15 =:0 <:-10) 
          <:[3·6·11·14·17·19·25 ⚖ 1·7·9·16·18·21·24] ➔ >:-16 =:+13 <:-14)) 
     <:[4·7·9·10·11·12·21·23·25 ⚖ 1·5·14·15·17·18·19·22·24] ➔ 
          >:[3·14·18·19·21·22 ⚖ 4·7·9·11·23·24] ➔ >:+21 =:+25 <:+23) 
          =:[1·2·10·11·13·15·23·25 ⚖ 3·5·6·12·16·18·19·24] ➔ >:-6 =:-8 <:-2) 
          <:[2·3·7·9·11·12·22·24 ⚖ 1·4·10·14·19·20·23·25] ➔ >:-4 =:+17 <:+19)))


# 4 Weighings

We can tackle larger puzzles. With 4 weighings, we can theoretically handle up to 3<sup>4</sup> = 81 possibilities. Can we solve for **40 balls**?

In [27]:
do(Puzzle(40, 4))

None


Unfortunately, **no**. How about **39 balls**, and how about we allow for the possibility that no ball is odd. That's 39 &times; 2 + 1 = 79 possibilities.

In [28]:
do(Puzzle(39, 4, {-1, 0, +1}))

[1·2·3·4·5·6·7·8·9·10·11·12·13 ⚖ 27·28·29·30·31·32·33·34·35·36·37·38·39] ➔ 
     >:[7·9·10·13·14·16·26·28·30·33·34 ⚖ 4·5·6·12·17·23·27·31·35·36·37] ➔ 
          >:[1·2·5·10·12·16·18·25·32·35·36 ⚖ 6·9·15·21·22·23·27·30·33·37·39] ➔ 
               >:[1·10·11·19·22·27·31 ⚖ 9·15·25·26·28·30·36] ➔ >:+10 =:-37 <:-27) 
               =:[12·13·15·26·27·29·31·35·36 ⚖ 3·4·6·9·14·23·32·37·38] ➔ >:+13 =:+7 <:-31) 
               <:[1·4·7·22·32·33·35·37 ⚖ 10·14·21·23·27·29·31·36] ➔ >:-36 =:+9 <:-35)) 
          =:[1·4·8·9·12·13·17·25·27·29·36 ⚖ 2·3·7·15·16·20·22·28·31·34·38] ➔ 
               >:[3·13·17·18·19·22·24·25·29·39 ⚖ 4·5·8·16·20·21·28·30·36·38] ➔ >:-38 =:+1 <:+8) 
               =:[5·7·13·16·21·23·24·26·27·30·31·32·35 ⚖ 2·9·10·12·14·15·17·19·25·36·37·38·39] ➔ >:-39 =:+11 <:-32) 
               <:[2·7·8·28 ⚖ 3·12·14·27] ➔ >:+2 =:-29 <:+3)) 
          <:[8·11·12·14·20·24·26·31·34·36·37·38 ⚖ 4·6·15·16·17·22·23·27·29·30·33·35] ➔ 
               >:[2·8·9·10·14·15·18·20·22·23·25·29·39 ⚖ 3·4·6·11

# 5 Weighings

With 5 weighings, 3<sup>5</sup> = 243, so I might be able to handle up to 121 lighter-or-heavier balls (242 possibilities). Let's try:

In [29]:
do(Puzzle(121, 5))

None


That didn't work. Let's back off to 2 &times; 120 + 1 = 241 possibilities:

In [30]:
do(Puzzle(120, 5, {-1, 0, +1}))

[1·2·3·4·5·6·7·8·9·10·11·12·13·14·15·16·17·18·19·20·21·22·23·24·25·26·27·28·29·30·31·32·33·34·35·36·37·38·39·40 ⚖ 81·82·83·84·85·86·87·88·89·90·91·92·93·94·95·96·97·98·99·100·101·102·103·104·105·106·107·108·109·110·111·112·113·114·115·116·117·118·119·120] ➔ 
     >:[2·3·6·11·12·13·25·27·29·36·38·40·42·43·44·45·50·52·54·56·60·62·69·76·78·79·81·82·83·84·85·87·91·95·99·100·104·119 ⚖ 1·4·7·14·17·18·20·21·22·30·31·32·35·37·39·47·48·55·59·64·66·70·73·77·88·89·90·101·103·105·106·108·109·111·112·114·117·120] ➔ 
          >:[3·5·10·12·13·18·27·28·36·37·46·52·57·61·64·67·75·80·82·85·91·94·95·97·100·102·103·108·109·112·116 ⚖ 1·2·4·6·7·9·20·26·29·33·40·43·51·54·55·56·63·65·68·70·71·72·73·87·88·90·96·104·105·111·119] ➔ 
               >:[2·9·17·20·27·29·38·41·43·45·49·51·52·54·55·61·65·78·81·84·85·88·90·100·104·108·112·114·116 ⚖ 1·6·7·12·14·16·28·30·33·35·42·56·58·59·64·71·72·77·80·82·93·98·102·105·106·107·109·111·119] ➔ 
                    >:[4·6·9·10·17·18·22·27·31·32·34·36·39·40·43·44·51·54·58·

That works (although the output is not easy to read).

Or, I could solve a puzzle with 243 possibilites&mdash;242 lighter-only balls, plus the possibility that no ball is lighter:

In [31]:
do(Puzzle(242, 5, {-1, 0}))

[1·2·3·4·5·6·7·8·9·10·11·12·13·14·15·16·17·18·19·20·21·22·23·24·25·26·27·28·29·30·31·32·33·34·35·36·37·38·39·40·41·42·43·44·45·46·47·48·49·50·51·52·53·54·55·56·57·58·59·60·61·62·63·64·65·66·67·68·69·70·71·72·73·74·75·76·77·78·79·80·81 ⚖ 162·163·164·165·166·167·168·169·170·171·172·173·174·175·176·177·178·179·180·181·182·183·184·185·186·187·188·189·190·191·192·193·194·195·196·197·198·199·200·201·202·203·204·205·206·207·208·209·210·211·212·213·214·215·216·217·218·219·220·221·222·223·224·225·226·227·228·229·230·231·232·233·234·235·236·237·238·239·240·241·242] ➔ 
     >:[3·6·12·16·22·23·24·25·29·33·39·40·42·43·44·45·51·52·54·66·68·70·78·80·81·87·89·90·92·94·97·99·101·103·107·110·112·113·120·125·127·137·141·144·146·152·155·157·158·165·166·169·173·174·175·176·180·183·184·192·193·194·197·201·212·213·216·217·218·224·228·229·230·235·237·240 ⚖ 2·7·9·13·19·20·21·26·30·31·32·38·41·48·49·53·56·58·64·67·69·71·74·75·76·77·82·84·86·88·95·100·105·109·111·114·115·116·126·128·129·133·134·139·142·145·147·1

# What's Next?

- What other puzzles can you solve?
- Can you make a table of solvable and unsolvable puzzles?
- What happens when it is a possibility that *two or more* balls are odd?
- Can you prove which puzzles are unsolvable? Can you modify the `good_partitions` function to be exhaustive (but not redundant, and still efficient)?
- Can you find trees that minimize the *mean* number of weighings, rather than minimizing the *maximum* number of weighings?
- What else can you discover?
